# 1. Библиотеки и настройки сессии

### Переменные сессии

In [20]:
DATA_PATH = "/labs/laba01/ml-100k/"
FILM_VALUES = "u.data"
FILM_LIST = "u.item"

### Управляющие объекты

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf()
conf.set("spark.app.name", "ivan.strazov - lab01")
sc = SparkContext(conf=conf)

In [47]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=conf).appName("ivan.strazov - lab01").getOrCreate()

# 2. Загрузка данных

In [4]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2020-09-05 20:38 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2020-09-05 20:38 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2020-09-05 20:38 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2020-09-05 20:38 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2020-09-05 20:38 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2020-09-05 20:38 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2020-09-05 20:38 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2020-09-05 20:38 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2020-09-05 20:38 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2020-09-05 20:38 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2020-09-05 20:38 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2020-09-05 20:38 /labs/laba01/ml-100k/u2.base


### Список фильмов

In [5]:
!hdfs dfs -ls /labs/laba01/ml-100k/u.item

-rw-r--r--   3 hdfs hdfs     236344 2020-09-05 20:38 /labs/laba01/ml-100k/u.item


In [21]:
film_list = sc.textFile(DATA_PATH + FILM_LIST)

In [22]:
film_list.take(10)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0',
 '6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0',
 '8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title

### Оценки фильмов

In [12]:
!hdfs dfs -ls /labs/laba01/ml-100k/u.data

-rw-r--r--   3 hdfs hdfs    1979173 2020-09-05 20:38 /labs/laba01/ml-100k/u.data


user id | item id | rating | timestamp

In [62]:
from pyspark.sql.types import StructType, StructField, IntegerType

In [63]:
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("rating", IntegerType()),
    StructField("timestamp", IntegerType())
])

In [64]:
df = spark.read \
        .schema(schema)\
        .format("csv")\
        .option("sep", "\t")\
        .load(DATA_PATH + FILM_VALUES)
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



# 3. Расчёт частот

In [ ]:
hist_film = df.filter(df.item_id == 318).groupBy("rating").count().orderBy("rating").select("count").collect()
hist_all = df.groupBy("rating").count().orderBy("rating").select("count").collect()

In [ ]:
hist_film = list(map(lambda x: x[0], hist_film))
hist_all = list(map(lambda x: x[0], hist_all))

# 4. Запись json

In [79]:
import json

In [82]:
dct = {
    "hist_film": hist_film,
    "hist_all": hist_all
}

with open("lab01.json", "w") as file:
    json.dump(dct, file)

In [83]:
sc.stop()